# First LIGO contracts on a local network

## Setup

In [1]:
import pytezos as tz

In this notebook, we're going to use a local Tezos network, using the Flextesa sandbox. To do so, we use the following `docker-compose.yml` file:

In [2]:
!cat docker-compose.yml

version: "3.6"
services:
  flextesa:
    container_name: my_flextesa
    restart: always
    image: oxheadalpha/flextesa:20230607
    command: nairobibox start
    environment:
      - block_time=4
      - flextesa_node_cors_origin=*
    ports:
      - 127.0.0.1:20000:20000
    expose:
      - 20000/tcp


Several [test networks are available for Tezos](https://tezos.gitlab.io/introduction/test_networks.html). A permanent one, called Ghostnet, is also used to deploy new releases of Tezos one week before deploying them on Mainnet. In later notebooks, we'll see how to use it.

For now, let's just use our local Flextesa instance. You can boot Flextesa by running `docker compose up -d` in your terminal.

## Communicating with the chain

In [3]:
TEZOS_RPC="http://localhost:20000"

 It is available on port 20000, and has a predefined account with a large amount of tez available for testing purpose. This account's address finishes by `jcjb` and is well-known among Tezos developers, who have the habit of calling it "alice".

In [4]:
import pytezos as tz
import time

alice = tz.Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")

In [5]:
alice.public_key_hash()

'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'

PyTezosClient instances are connections to a given network, to which they can make request. They can also have a key, which makes it easy to query information about a given account and to make transactions. Let's create a connection to our local network and query Alice's balance.

In [6]:
ptz = tz.pytezos.using(TEZOS_RPC, alice)
ptz.balance()

Decimal('1999999.684358')

For security and determinism reasons, Tezos native numbers are integers only (naturals and integers), no floating-poing numbers. Currencies (tez and tokens alike) are expected to have a known precision; for tez, this precision is $10^{-6}$.

Thus, to represent 1.5tez, one can write 1500000mutez. Within PyTezos, the type `int` is used to represent a quantity of mutez, but `Decimal` is also available and more convenient. Here, Alice has 2M tez (the actual quantity may vary a bit).

Here's an example of transaction sending `1e6`mutez (= 1tez):

In [7]:
ptz.transaction(
    destination="tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu",
    amount=int(1e6)
).autofill().sign().inject()

{'chain_id': 'NetXNQiqPWDBoxN',
 'hash': 'ooU1mqnzcgLeUQLPANCCPVvgmDYezJ8NRCTDnPCPuDkvWi8neqx',
 'protocol': 'PtNairobiyssHuh87hEhfVBGCVrK3WnS8Z2FT4ymB5tAa4r1nQf',
 'branch': 'BM5RrVG3UcNzXTtFZxmzwYqGrruegSFEbCykwV2XeqSUfvLRm8M',
 'contents': [{'kind': 'transaction',
   'source': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
   'fee': '287',
   'counter': '13',
   'gas_limit': '269',
   'storage_limit': '357',
   'amount': '1000000',
   'destination': 'tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu'}],
 'signature': 'sigmMewC4SkkE56FwLv26nSsffmtYjju4pb2fxztLzYXhyeSg5XtWG4v8QtyMvqzTFAXgxw53AEFH5cN2VDWRed3zwgH6Z5f'}

This Python code does not wait for the transaction's inclusion, and returns immediately. The return `dict` contains various information about the transaction, including it's hash and signature. Try to send it a few times, and then look how it changed Alice's balance.

To wait for inclusion, add the `min_confirmations=1` parameter.

In [9]:
ptz.transaction(
    destination="tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu",
    amount=int(1e6)
).autofill().sign().inject(min_confirmations=1)

{'protocol': 'PtNairobiyssHuh87hEhfVBGCVrK3WnS8Z2FT4ymB5tAa4r1nQf',
 'chain_id': 'NetXNQiqPWDBoxN',
 'hash': 'oohhGDETdroLqLJTsP7gqudVpXTGVSiSjS9DPSkGHY796yYTPQs',
 'branch': 'BMVw8DBnkn6kP4LAWEsx3xTm5t1fV1Wt8t3RRzen9tgnax7goDb',
 'contents': [{'kind': 'transaction',
   'source': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
   'fee': '286',
   'counter': '14',
   'gas_limit': '269',
   'storage_limit': '100',
   'amount': '1000000',
   'destination': 'tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu',
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
      'change': '-286',
      'origin': 'block'},
     {'kind': 'accumulator',
      'category': 'block fees',
      'change': '286',
      'origin': 'block'}],
    'operation_result': {'status': 'applied',
     'balance_updates': [{'kind': 'contract',
       'contract': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
       'change': '-1000000',
       'origin': 'block'},
      {'kind': 'contract',


Just as the `octez-client` command line client, PyTezos provides an interface to the RPC node, which in turns relies our transactions to the Tezos network. RPC paths such as `rpc_address/a/b/c` are translated by chained Python calls, such as `ptz.a.b.c()`. For instance, you can get all the contracts known by a RPC by doing:

In [10]:
ptz.shell.contracts()

['KT1VqarPDicMFn1ejmQqqshUkUXTCTXwmkCN',
 'KT1TxqZ8QtKvLu3V3JH7Gx58n7Co8pgtpQU5',
 'KT1TetEit2fWULZMtu4GMLXfxuWY8qDf9pnU',
 'KT1REvYuUpuKuiBbExk1JFQjSp3taN4S14r2',
 'KT1CQT2h8tdHajt7CY6GDYhjwV6jZvSFwXUR',
 'KT1C49Dx86roc1mY4zXh6GNaP9T8Qk8Rw9ft',
 'KT1AafHA1C1vk959wvHWBispY9Y2f3fxBUUo',
 'tz1aYQcaXmowUu59gAgMGdiX6ARR7gdmikZk',
 'tz1aSkwEot3L2kmUvcoxzjMomb9mvBNuzFK6',
 'tz1YPSCGWXwBdTncK2aCctSZAXWvGsGwVJqU',
 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
 'tz1SEQPRfF2JKz7XFF3rN2smFkmeAmws51nQ',
 'tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu']

And here is how you can provide arguments to a RPC call:

In [11]:
ptz.shell.contracts["tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu"]()

{'balance': '2000000', 'counter': '13'}

Note: the word "contract" is used a bit loosely. Tezos differentiates between
* implicit accounts, which addresses start with `tz`, and which are defined by a secret key. They also can sign transactions.
* originated accounts, which addresses start with `KT`, and which we will call "smart contracts" most of the time.

# Deploying a smart contract

In this section we're going to write our first smart contract using the LIGO language, and originate it on our local network. If you're not interested in learning LIGO, you can skip the first section safely.

## First smart contract

We're going to write and compile our smart contract directly from this notebook.

In [12]:
%%writefile first-contract.mligo

type storage = unit

[@entry]
let main (_ : unit) (store : storage) : operation list * storage =
    ([], store)

Overwriting first-contract.mligo


This may look a bit complicated, and yet this is this the simplest contract you can write in LIGO. Here's what it does:

First, it defines a type named `storage` as a synonym to the `unit` type. `unit` is the simplest type available in LIGO and Michelson: it only contains only one value, written `()` in the Cameligo syntax. You can think of it as a tuple with no elements (a tuple of length 0).

Then, it defines a single entrypoint named `main`, which takes two arguments: one of type `unit`, and one of type `storage`. It also returns a couple of types, `operation list` and `storage`. Note that tuples _types_ are defined with a `*`, which should remind you of the mathematical notation for the Cartesian product between sets. When you write `type_1 * type_2` in Cameligo, you're really writing "the type which contains elements which are pairs of `type_1` and `type_2`".

In LIGO, all entrypoints have to
- take, as a first argument, the data that the entrypoint requires;
- take, as second argument, the contract's storage: in this variable (called `store` here), the entrypoint receives the value of the contract's storage when the entrypoint is called. If anything is read from the storage, it has to be explicitely read from this variable;
- return a couple containing the list of operations that the contract generates (such as transfers or calls to other smart contracts — or itself!), and the new value of the storage. Again, if anything is written in the storage, this has to be explicitely returned here.

For our first contract, we don't want our entrypoint to do anything, hence it does not need any particular data as first argument. The `unit` type does just fine! To match the type requirements for the entrypoint's return, we return a couple, made of the empty list and the store as we found it.

To compile the contract, let's call the LIGO compiler:

In [13]:
! ligo compile contract first-contract.mligo

{ parameter unit ; storage unit ; code { CDR ; NIL operation ; PAIR } }



With such a small example, the Michelson code is almost easier to read than the original LIGO code. As you can see, it defines a contract which expects a `unit` parameter and has a `unit` storage.

Michelson is a stack-based language: its instructions handle the data on a stack, where they push, pop or duplicate values. When a call to this smart contract is executed, Tezos starts by putting the parameter and the storage on the stack, as a couple. Then the code does the following:
* `CDR` takes the second part of the couple (which is `()`, the unit value);
* `NIL operation` pushes an empty list of type `operation` on the stack;
* `PAIR` builds the tuple with this list and the unit value, the latter being the final value for our storage.

You don't have to understand Michelson to write smart contracts on Tezos. However, the simplicity of this example gives us the opportunity to take a look and understand how it works.

Let's write this contract to a file, and read it back from Python to originate it.

In [14]:
! ligo compile contract first-contract.mligo > first-contract.tz

## Originating our first contract

We'll need the following class to read the contract:

In [15]:
from pytezos import ContractInterface

PyTezos leverages Python's documentation strings to provide information about the arguments of the methods, contract calls, etc. that you may want to use.

To deploy a contract, use the `origination` method:

In [16]:
ptz.origination

<function ContentMixin.origination at 0x7f11da64fb00>
Deploy smart contract (scriptless KT accounts are not used for delegation since Babylon).

:param script: {"code": $Micheline, "storage": $Micheline}
:param balance: Amount transferred on the balance, WARNING: there is no default way to withdraw funds.
More info: https://tezos.stackexchange.com/questions/1315/can-i-withdraw-funds-from-an-empty-smart-contract
:param delegate: Set contract delegate, default None
:param source: Address from which funds will be sent, leave None to use signatory address
:param counter: Current account counter, leave None for autocomplete
:param fee: Leave None for autocomplete
:param gas_limit: Leave None for autocomplete
:param storage_limit: Leave None for autocomplete
:returns: dict or OperationGroup

That's a lot of parameters! However, not all of them are required. The most important one is the `script` of the contract, which is its code and original storage value. Let's load the file `first-contract.tz` and originate it.

In [17]:
first_contract = ContractInterface.from_file("first-contract.tz")

Here again, the generated docuemntation is really helpful. Our contract's entrypoint is even listed. Note that its name was changed: when a LIGO contract with only one entrypoint is compiled, its name does not appear in the resulting contract. PyTezos uses the name `default` for this.

In [18]:
first_contract


Properties
.block_id	head
.storage	# access storage data at block `block_id`
.parameter	# root entrypoint

Entrypoints
.default()

Views

Helpers
.big_map_get()
.create_from()
.from_context()
.from_file()
.from_micheline()
.from_michelson()
.from_url()
.metadata()
.metadata_url()
.operation_result()
.originate()
.program()
.script()
.storage_from_file()
.storage_from_micheline()
.storage_from_michelson()
.to_file()
.to_micheline()
.to_michelson()
.using()

In [19]:
origination = ptz.origination(first_contract.script()).autofill().sign().inject(min_confirmations=1)

In [20]:
origination

{'protocol': 'PtNairobiyssHuh87hEhfVBGCVrK3WnS8Z2FT4ymB5tAa4r1nQf',
 'chain_id': 'NetXNQiqPWDBoxN',
 'hash': 'onsSDGuxnbSmcsFhKLoq4faNDKiMSKdZL7Hw4vuwFGvUPBPLWbF',
 'branch': 'BL8oA6wfSEHsbBi5uuGcpcjdphAcxMaTUUeJJbLckhZCGH7Fafz',
 'contents': [{'kind': 'origination',
   'source': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
   'fee': '342',
   'counter': '15',
   'gas_limit': '679',
   'storage_limit': '395',
   'balance': '0',
   'script': {'code': [{'prim': 'parameter', 'args': [{'prim': 'unit'}]},
     {'prim': 'storage', 'args': [{'prim': 'unit'}]},
     {'prim': 'code',
      'args': [[{'prim': 'CDR'},
        {'prim': 'NIL', 'args': [{'prim': 'operation'}]},
        {'prim': 'PAIR'}]]}],
    'storage': {'prim': 'Unit'}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
      'change': '-342',
      'origin': 'block'},
     {'kind': 'accumulator',
      'category': 'block fees',
      'change': '342',
      'origin': 'bl

Again, lots of information here. What we need, though, is pretty simple: to interact with our contract, we want to get the address to which it was originated. You can either copy it from all of this, or get it with the following expression:

In [21]:
contract_address = origination['contents'][0]['metadata']['operation_result']['originated_contracts'][0]

In [22]:
contract_address

'KT1PtkWbWAEpjD2swLzkSKpJpHpuDafem6CK'

Now that we have an address, let's get the on-chain contract object:

In [23]:
contract = ptz.contract(contract_address)

We can access various properties from the contract, such as its storage:

In [24]:
contract.storage()

Unit

Are we walking in circle? No, this contract is a different one from the `first_contract` we made before. As we got it from a `PyTezosClient`, we have an address for it (you can compare `first_contract.address` and `contract.address`). Moreover, if we try to call its entrypoint, the transactions is going to get signed automatically by `alice`.

Let's use the documentation one more time to see what parameters are expected:

In [25]:
contract.default


Properties
.key		tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb
.shell		['http://localhost:20000']
.address	KT1PtkWbWAEpjD2swLzkSKpJpHpuDafem6CK
.block_id	head
.entrypoint	default

Builtin
(*args, **kwargs)	# build transaction parameters (see typedef)

Typedef
$default:
	unit

$unit:
	None || Unit /* `from pytezos import Unit` for resolving None ambiguity */


Helpers
.decode()
.encode()

Let's send a transaction calling this entrypoint:

In [26]:
op = contract.default().send(min_confirmations=1)
op.contents

[{'kind': 'transaction',
  'source': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb',
  'fee': '395',
  'counter': '16',
  'gas_limit': '1378',
  'storage_limit': '100',
  'amount': '0',
  'destination': 'KT1PtkWbWAEpjD2swLzkSKpJpHpuDafem6CK'}]

Of course, the storage did not change, just as we wrote it:

In [27]:
contract.storage()

Unit

# A better example

To conclude this notebook, let's write a contract that stores an `int`, and offers two entrypoints to either increment or decrement this `int`.

In [28]:
%%writefile counter.mligo

type storage = int

[@entry]
let increment (_ : unit) (store : storage) : operation list * storage =
    ([], store + 1)
    
[@entry]
let decrement (_ : unit) (store : storage) : operation list * storage =
    ([], store - 1)

Overwriting counter.mligo


In [29]:
!ligo compile contract counter.mligo > counter.tz

In [30]:
counter_contract = ContractInterface.from_file("counter.tz")

In [31]:
origination = ptz.origination(counter_contract.script()).autofill().sign().inject(min_confirmations=1)

In [32]:
contract_address = origination['contents'][0]['metadata']['operation_result']['originated_contracts'][0]

In [33]:
counter_contract = ptz.contract(contract_address)

In [34]:
counter_contract


Properties
.key		tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb
.shell		['http://localhost:20000']
.address	KT1TbLGRc3Fz9ihonAonRQ2P5phedNfPdbP2
.block_id	head
.storage	# access storage data at block `block_id`
.parameter	# root entrypoint

Entrypoints
.decrement()
.increment()
.default()

Views

Helpers
.big_map_get()
.create_from()
.from_context()
.from_file()
.from_micheline()
.from_michelson()
.from_url()
.metadata()
.metadata_url()
.operation_result()
.originate()
.program()
.script()
.storage_from_file()
.storage_from_micheline()
.storage_from_michelson()
.to_file()
.to_micheline()
.to_michelson()
.using()

The initial storage has automatically be set to 0. 

In [35]:
counter_contract.storage()

0

In [36]:
op1 = counter_contract.increment().send(min_confirmations=1)
op2 = counter_contract.increment().send(min_confirmations=1)

In [37]:
counter_contract.storage()

2

The contract storage has correctly been updated.

# To go further…

* You can find an introduction to LIGO on the official page: https://ligolang.org/docs/intro/introduction?lang=cameligo . In this series of notebooks, we'll explain the concepts we need, but you should refer to this official website to learn more about the language.
* Interact with `counter_contract` without waiting for confirmations and immediately get the value from the storage. What can you observe?
* Using `from pytezos import Key`, generate a new address and the corresponding PyTezos client. You will need to send it money from `alice`, and post a `new_ptz.reveal().autofill().sign().inject()` before you can use it ([more information about reveal operations](https://tezos.stackexchange.com/questions/786/what-is-the-adequate-fee-for-a-reveal-operation)).
* Modify the counter contract to only accept increments that come from `alice` and originate it again. You can make the contract fail using the `failwith` instructions. Test your contract using the other addresses you generated.